# JupyerLab notebook example for UPENN like simulations

/!\ /!\ /!\  
Oxygen diffusion from cell environment not (yet) included in the model.  
Therefore, the oxygen level is reinitialized before each new pulse.  

## Module imports

In [ ]:
# External modules (to "pip install")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Modules from Standard lib
from pathlib import Path

# RadioBio module
import radiopyo as rp

## Simulation Run

**1) Load configuration from the toml file +**  
 -> create a ref to O2 species for later use  
 -> define the desired number of pulses  
 -> define the period

In [ ]:
file = Path(rf"configuration_UPENN.toml")
uc = rp.UnitCell.from_toml(file)

O2 = uc.env.species.get("O2")
PULSES_NUMBER = 3
PERIOD = 120 # seconds

**2) Run sim over the first pulse**

In [ ]:
res = uc.prepare_chunked_run([1e-9, PERIOD],
                             max_step_size_on=1e-5,
                             max_step_size_off=0.5).run()

**3) Run the next pulses**
* Reset beam start time  
* Copy final concentrations of previous sim  
* Reset the O2 concentration to its initial value (in µmol/L)  
* Run and append the simulation result to the previous result  

In [ ]:
for pulse in range(PULSES_NUMBER-1):
    uc.beam.reset(start=res.time[-1])
    y0 = res.final_cc
    y0[O2.index] = O2.initial_cc()*1e6
    res +=  uc.prepare_chunked_run([res.time[-1], res.time[-1]+PERIOD],
                                   max_step_size_on=1e-5,
                                   max_step_size_off=0.5,
                                   y0=y0).run()

**Convert final result to pandas DataFrame**

In [ ]:
df = res.to_pandas()

## Results usage

**Integrate values per pulse**

In [ ]:
by_pulse = pd.DataFrame(columns=df.columns)
for pulse in range(PULSES_NUMBER):
    by_pulse.loc[pulse, :] = res.integrate_species(start=pulse*PERIOD, stop=(pulse+1)*PERIOD)

### Plot

In [ ]:
# Some common plot options for uniformity
RIGHT_LIMIT = 1e3

#### Water-only Radiolytic produced species

In [ ]:
G_species = ["OH_r", "e_aq", "H2O2", "H_r"]
fig, ax = plt.subplots()
for label in G_species:
    ax.plot(df.index, df[label], label=label, marker="")

ax.set_ylim(bottom=1e-6, top=10)
ax.set_xlim([1e-6, RIGHT_LIMIT])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Concentration [µmol/L]")
ax.legend(fancybox=True, framealpha=1)
ax.set_title("Water radiolysis related species")
ax.grid()

#### Biology related species

In [ ]:
G_species = ["ROO_r", "R_r"]
fig, ax = plt.subplots()
for label in G_species:
    ax.plot(df.index, df[label], label=label, marker="")

ax.set_ylim(bottom=1e-6, top=100)
ax.set_xlim([1e-6, RIGHT_LIMIT])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Concentration [µmol/L]")
ax.legend(fancybox=True, framealpha=1)
ax.set_title("Biology related species")
ax.grid()

#### O2

In [ ]:
G_species = ["O2"]
fig, ax = plt.subplots()
for label in G_species:
    ax.plot(df.index, df[label], label=label, marker="")

ax.set_ylim(bottom=40, top=52)
ax.set_xlim([1e-6, RIGHT_LIMIT])
ax.set_xscale("log")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Concentration [µmol/L]")
ax.set_title("Molecular Oxygen")
ax.grid()